In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.svm import SVC
import spacy
!pip install sklearn-crfsuite
import sklearn_crfsuite
from sklearn_crfsuite import metrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 12.2 MB/s eta 0:00:00


In [3]:
import pandas as pd
import zipfile
zf = zipfile.ZipFile('/content/drive/MyDrive/COMP723Data/FakeReal.zip')
fake_news = pd.read_csv(zf.open('Fake.csv'))
true_news = pd.read_csv(zf.open('True.csv'))

In [ ]:
# Counting the number of missing values in each column of the 'true_news' dataset.
true_news.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [ ]:
# Counting the number of missing values in each column of the 'fake_news' dataset.
fake_news.isnull().sum()

title      0
text       0
subject    0
date       0
dtype: int64

In [ ]:
# Getting the shape of the 'true_news' dataset.
true_news.shape

(21417, 4)

In [ ]:
# Getting the shape of the 'fake_news' dataset.
fake_news.shape

(23481, 4)

In [ ]:
# Displaying the first 5 rows of the 'true_news' dataset.
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [ ]:
# Displaying the first 5 rows of the 'fake_news' dataset.
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [ ]:
# Adding a new column called 'label' to the 'true_news' dataset and assigning the value 'True' to all rows.
true_news["label"] = 'True'

# Adding a new column called 'label' to the 'fake_news' dataset and assigning the value 'Fake' to all rows.
fake_news["label"] = 'Fake'

In [ ]:
# Concatenating/combining the 'fake_news' and 'true_news' datasets into a single dataset called 'data'.
data = pd.concat([fake_news,true_news],ignore_index=True)

In [ ]:
# Displaying the first 5 rows of the 'data' dataset.
data.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",Fake
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",Fake
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",Fake
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",Fake
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",Fake


In [ ]:
data.shape

(44898, 5)

In [ ]:
from sklearn.model_selection import train_test_split # This function allows the splitting of a dataset into training and testing sets.

X = data["text"] # Selecting the 'text' column from the 'data' dataset as the features (X)
y = data["label"] # Selecting the 'label' column from the 'data' dataset as the labels (y)

# Spliting the features and labels into training and testing sets.
# The training set will contain 80% of the data and the test set will contain the remaining 20%
# The random state is set to 42 to ensure the splits are reproducible
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Creating a token frequency counter
counter = CountVectorizer()

# Transforming the training data into a matrix of token counts
X_train_matrix = counter.fit_transform(X_train)

# Transforming the test data into a matrix of token counts
X_test_matrix = counter.transform(X_test)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Training the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_matrix, y_train)

# Making predictions
y_pred = clf.predict(X_test_matrix)

# Evaluating the model
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9522271714922049


In [ ]:
from sklearn.neural_network import MLPClassifier

# Training the Neural Network (MLP) classifier
clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
clf.fit(X_train_matrix, y_train)

# Making predictions
y_pred = clf.predict(X_test_matrix)

# Evaluating the model
print("Accuracy:", accuracy_score(y_test, y_pred))

Accuracy: 0.9879732739420936


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Defining the Decision Tree model
model = DecisionTreeClassifier()

# Training the model
model.fit(X_train_matrix, y_train)

# Making predictions
y_pred = model.predict(X_test_matrix)

# Evaluating the model
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9965478841870824


In [ ]:
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

        Fake       0.99      0.99      0.99      4733
        True       0.99      0.99      0.99      4247

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB

param_grid = {
    'alpha': [0.1, 0.5, 1, 1.5, 2]
}

model = MultinomialNB()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

grid_search.fit(X_train_matrix, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)

Best parameters:  {'alpha': 0.1}
Best score:  0.9556489804848898


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],
    'activation': ['relu', 'tanh', 'logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.001, 0.01, 0.1],
    'learning_rate_init': [0.001, 0.01, 0.1]
}

model = MLPClassifier(max_iter=1000, random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=2)


grid_search.fit(X_train_matrix, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


KeyboardInterrupt: ignored

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid = {
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10, 20]
}

model = DecisionTreeClassifier()
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, scoring='accuracy')

grid_search.fit(X_train_matrix, y_train)

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", grid_search.best_score_)


Best parameters:  {'max_depth': 9, 'min_samples_split': 5}
Best score:  0.9951835005902532


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

chosen_category = 'True'

# Calculating precision
precision = precision_score(y_test, y_pred, pos_label=chosen_category)

# Calculating recall
recall = recall_score(y_test, y_pred, pos_label=chosen_category)

# Calculating F1 score
f1 = f1_score(y_test, y_pred, pos_label=chosen_category)

print(f"Precision for {chosen_category}: {precision}")
print(f"Recall for {chosen_category}: {recall}")
print(f"F1 score for {chosen_category}: {f1}")


Precision for True: 0.9950681070925317
Recall for True: 0.9976453967506476
F1 score for True: 0.9963550852439741


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords

# Defining the stop words
stop_words = 'english'  # Use the string 'english'

# Creating a token frequency counter that doesn't count stop words
counter = CountVectorizer(stop_words=stop_words)

# Transforming the training data into a matrix of token counts
X_train_matrix = counter.fit_transform(X_train)

# Transforming the test data into a matrix of token counts
X_test_matrix = counter.transform(X_test)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# Pre-processing using CountVectoriser
vectorizer = CountVectorizer(
    lowercase=True,           # Converting text to lowercase
    stop_words='english',     # Removing common English stopwords
    max_features=5000         # Limiting the number of features
)

# Vectorising the data
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Training the Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

# Making predictions
y_pred = clf.predict(X_test_vectorized)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9443207126948775


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Pre-processing using CountVectoriser
vectorizer = CountVectorizer(
    lowercase=True,           # Converting text to lowercase
    stop_words='english',     # Removing common English stopwords
    max_features=5000         # Limiting the number of features
)

# Vectoriseing the data
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Training Decision Tree classifier
clf = DecisionTreeClassifier()
clf.fit(X_train_vectorized, y_train)

# Making predictions
y_pred = clf.predict(X_test_vectorized)

# Evaluating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.9946547884187082


In [ ]:
data.shape

(44898, 5)